# Turing Target Practice

## Model

In [1]:
using Pkg
Pkg.activate(".")
Pkg.develop(path="../")
Pkg.instantiate()

  Activating project at `~/PhD/MicroCanonicalHMC.jl/test`
   Resolving package versions...
  No Changes to `~/PhD/MicroCanonicalHMC.jl/test/Project.toml`
  No Changes to `~/PhD/MicroCanonicalHMC.jl/test/Manifest.toml`


In [2]:
# The statistical inference frame-work we will use
using Turing
using Interpolations
using ForwardDiff
using LinearAlgebra
#using StatsPlots
using PyPlot
using Distributed

using Revise
using MicroCanonicalHMC

[ Info: Precompiling MicroCanonicalHMC [234d2aa0-2291-45f7-9047-6fa6f316b0a8]


In [3]:
fs8_zs = [0.38, 0.51, 0.61, 1.48, 0.44, 0.6, 0.73, 0.6, 0.86, 0.067, 1.4]
fs8_data = [0.49749, 0.457523, 0.436148, 0.462, 0.413, 0.39, 0.437, 0.55, 0.4, 0.423, 0.482]
fs8_cov = [0.00203355 0.000811829 0.000264615 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
           0.000811829 0.00142289 0.000662824 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 
           0.000264615 0.000662824 0.00118576 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
           0.0 0.0 0.0 0.002025 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
           0.0 0.0 0.0 0.0 0.0064 0.00257 0.0 0.0 0.0 0.0 0.0;
           0.0 0.0 0.0 0.0 0.00257 0.003969 0.00254 0.0 0.0 0.0 0.0;
           0.0 0.0 0.0 0.0 0.0 0.00254 0.005184 0.0 0.0 0.0 0.0;
           0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0144 0.0 0.0 0.0;
           0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0121 0.0 0.0; 
           0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.003025 0.0;
           0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.013456000000000001];

In [4]:
function make_fs8(Ωm, σ8; Ωr=8.24*10^-5)
    # ODE solution for growth factor
    x_Dz = LinRange(0, log(1+1100), 300)
    dx_Dz = x_Dz[2]-x_Dz[1]
    z_Dz = @.(exp(x_Dz) - 1)
    a_Dz = @.(1/(1+z_Dz))
    aa = reverse(a_Dz)
    e =  @.(sqrt.(abs(Ωm)*(1+z_Dz)^3+Ωr*(1+z_Dz)^4+(1-Ωm-Ωr)))
    ee = reverse(e)

    dd = zeros(typeof(Ωm), 300)
    yy = zeros(typeof(Ωm), 300)
    dd[1] = aa[1]
    yy[1] = aa[1]^3*ee[end]

    for i in 1:(300-1)
        A0 = -1.5 * Ωm / (aa[i]*ee[i])
        B0 = -1. / (aa[i]^2*ee[i])
        A1 = -1.5 * Ωm / (aa[i+1]*ee[i+1])
        B1 = -1. / (aa[i+1]^2*ee[i+1])
        yy[i+1] = (1+0.5*dx_Dz^2*A0*B0)*yy[i] + 0.5*(A0+A1)*dx_Dz*dd[i]
        dd[i+1] = 0.5*(B0+B1)*dx_Dz*yy[i] + (1+0.5*dx_Dz^2*A0*B0)*dd[i]
    end

    y = reverse(yy)
    d = reverse(dd)

    Dzi = LinearInterpolation(z_Dz, d./d[1], extrapolation_bc=Line())
    fs8zi = LinearInterpolation(z_Dz, -σ8 .* y./ (a_Dz.^2 .*e.*d[1]),
                                 extrapolation_bc=Line())
    return fs8zi
end

make_fs8 (generic function with 1 method)

In [5]:
@model function model(data; cov = fs8_cov) 
    # Define priors
    #KiDS priors
    Ωm ~ truncated(Normal(0.3, 0.05), 0.01, 0.6)
    σ8 ~ Normal(0.8, 0.3)
    fs8_itp = make_fs8(Ωm, σ8)
    theory = fs8_itp(fs8_zs)
    data ~ MvNormal(theory, cov)
end;

In [6]:
stat_model = model(fs8_data)

DynamicPPL.Model{typeof(model), (:data, :cov), (:cov,), (), Tuple{Vector{Float64}, Matrix{Float64}}, Tuple{Matrix{Float64}}, DynamicPPL.DefaultContext}(model, (data = [0.49749, 0.457523, 0.436148, 0.462, 0.413, 0.39, 0.437, 0.55, 0.4, 0.423, 0.482], cov = [0.00203355 0.000811829 … 0.0 0.0; 0.000811829 0.00142289 … 0.0 0.0; … ; 0.0 0.0 … 0.003025 0.0; 0.0 0.0 … 0.0 0.013456000000000001]), (cov = [0.00203355 0.000811829 … 0.0 0.0; 0.000811829 0.00142289 … 0.0 0.0; … ; 0.0 0.0 … 0.003025 0.0; 0.0 0.0 … 0.0 0.013456000000000001],), DynamicPPL.DefaultContext())

## Sampling

In [7]:
target = TuringTarget(stat_model);

In [8]:
spl = MCHMC(10_000, 0.5; sigma=ones(target.d), adaptive=true)
samples_mchmc = Sample(spl, target, 100_000;
                       initial_x = [0.2, 0.8],  
                       dialog=true)

[ Info: Tuning eps ⏳
[ Info: Tuning L ⏳
MCHMC (tuning):   0%|                                   |  ETA: 0:43:53

Burn in step: 2000
eps --->0.09670701689535931


MCHMC (tuning):  20%|███████                            |  ETA: 0:00:06

L   --->0.012378403746625376
 


MCHMC (tuning):  30%|██████████▌                        |  ETA: 0:00:04

...

Burn in step: 8000
eps --->0.07953752743947605
L   --->0.01348665786128366
 


2mMCHMC (tuning): 100%|███████████████████████████████████| Time: 0:00:02

Burn in step: 10000
eps --->0.07944556442605132
L   --->0.013427649050369373
 


[ Info: eps: 0.07944556442605132
[ Info: L: 0.013427649050369373
[ Info: nu: 255.72458030926668
[ Info: sigma: [1.0, 1.0]
[ Info: adaptive: true
2mMCHMC: 100%|████████████████████████████████████████████| Time: 0:00:04

100000-element Vector{Any}:
 [0.31085155249296387, 0.8223922853859423, 0.07944556442605132, -0.00484918084685404, 17.058553381713633]
 [0.30895778184544015, 0.7439916328284029, 0.07993980765039024, 0.33979818284405994, 15.169038535763057]
 [0.3085702400217716, 0.8238882369780204, 0.080383384925388, -0.03688380306328476, 17.0982265655041]
 [0.3080794716959511, 0.7435737600225233, 0.08065022405210387, 0.003414853423503672, 15.139578715933636]
 [0.31781350071032743, 0.697289790065728, 0.0758262031887847, 3.4172338184307147, 11.843341154845508]
 [0.31847741397858875, 0.7729817474310026, 0.07632238966426556, -0.28447861855060097, 16.38976810007717]
 [0.3182818113552168, 0.8492925678542841, 0.07679615580316403, 0.07106693641684458, 16.305901941084954]
 [0.3282888708984768, 0.8138200927267825, 0.07711270277505167, -0.6597873069521505, 16.649902091903158]
 [0.3172947915834497, 0.8324956456742034, 0.07762549228120749, 0.18684110891687666, 16.757772178349907]
 [0.306147270206382, 0.8149849800425

In [9]:
std(samples_mchmc)[1:end-3]

2-element Vector{Float64}:
 0.03837056577659039
 0.053420217988547555

In [10]:
Wms_mchmc = [sample[1] for sample in samples_mchmc]
s8s_mchmc = [sample[2] for sample in samples_mchmc];

In [11]:
plt.hist2d(Wms_mchmc, s8s_mchmc, bins=100, range=[[0.1, 0.6],[0.6, 1.2]]);
plt.xlabel("Wm")
plt.ylabel("s8")
plt.title("MCHMC - RSD model");

## NUTS

In [12]:
samples_hmc = sample(stat_model, NUTS(1_000, 0.65),
                     50_000, progress=true; save_state=true)

┌ Info: Found initial step size
└   ϵ = 0.05
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:09


Chains MCMC chain (50000×14×1 Array{Float64, 3}):

Iterations        = 1001:1:51000
Number of chains  = 1
Samples per chain = 50000
Wall duration     = 13.3 seconds
Compute duration  = 13.3 seconds
parameters        = Ωm, σ8
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std      mcse     ess_bulk     ess_tail      rhat ⋯
      Symbol   Float64   Float64   Float64      Float64      Float64   Float64 ⋯

          Ωm    0.2748    0.0431    0.0003   24018.0261   27296.8275    1.0000 ⋯
          σ8    0.8350    0.0448    0.0003   25347.5199   28732.2265    1.0000 ⋯
                                                                1 column omitted

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

     

In [13]:
Wms_hmc = vec(samples_hmc["Ωm"])
s8s_hmc = vec(samples_hmc["σ8"]);

In [14]:
fig, axis = plt.subplots(2, 2, figsize=(8,8))
fig.suptitle("RSD's Comp.", fontsize=16)

fig.subplots_adjust(hspace=0)
fig.subplots_adjust(wspace=0)

axis[1,1].hist(Wms_hmc, bins=100, density=true, range=[0.1, 0.6], alpha = 0.3, label="NUTS")
axis[1,1].hist(Wms_mchmc, bins=100, density=true, range=[0.1, 0.6], alpha = 0.3, label="MCHMC")
axis[1,1].legend()
axis[1,1].set_yticks([])

axis[2,2].hist(s8s_hmc, bins=100, density=true, orientation="horizontal", range=[0.6, 1.2], alpha = 0.3)
axis[2,2].hist(s8s_mchmc, bins=100, density=true, orientation="horizontal", range=[0.6, 1.2], alpha = 0.3)
axis[2,2].set_xticks([])
axis[2,2].set_yticks([])

axis[1,2].hist2d(Wms_hmc, s8s_hmc, bins=100, range=[[0.1, 0.6],[0.6, 1.2]])
axis[1,2].set_xlabel("Wm")
axis[1,2].set_ylabel("s8")
axis[1,2].set_title("NUTS")

axis[2,1].hist2d(Wms_mchmc, s8s_mchmc, bins=100, range=[[0.1, 0.6],[0.6, 1.2]])
axis[2,1].set_xlabel("Wm")
axis[2,1].set_ylabel("s8")
axis[2,1].set_title("MCHMC")
;